In [5]:
#this worksheet contains different examples of how to use the raw data files for various one off analyses.


#importing packages needed for analysis
import pandas as pd
import numpy as np
import os


In [6]:
#Read in the raw datasets

eia923g_raw = pd.read_csv('outputs/eia923g_raw.csv', thousands=',', encoding='latin-1', low_memory=False)
eia923g_raw.columns = eia923g_raw.columns.str.replace('\n', ' ')
eia923g_raw.columns = eia923g_raw.columns.str.replace('  ', ' ')
eia923g_raw.columns = eia923g_raw.columns.str.replace('_', ' ')
#print(eia923g_raw.columns)

eia923p_raw = pd.read_csv('outputs/eia923p_raw.csv', thousands=',', encoding='latin-1', low_memory=False)
eia923p_raw.columns = eia923p_raw.columns.str.replace('\n', ' ')
eia923p_raw.columns = eia923p_raw.columns.str.replace('  ', ' ')
eia923p_raw.columns = eia923p_raw.columns.str.replace('_', ' ')
print(eia923p_raw.columns)

eia860_raw = pd.read_csv('outputs/eia860_raw.csv', thousands=',', encoding='latin-1', low_memory=False)
#print(eia860_raw.columns)

eia860_RE_raw = pd.read_csv('outputs/eia860_RE_raw.csv', thousands=',', encoding='latin-1', low_memory=False)
#print(eia860_RE_raw.columns)


Index(['Plant Id', 'Combined Heat And Power Plant', 'Nuclear Unit Id',
       'Plant Name', 'Operator Name', 'Operator Id', 'Plant State',
       'Census Region', 'NERC Region', 'Reserved', 'NAICS Code',
       'EIA Sector Number', 'Sector Name', 'Reported Prime Mover',
       'Reported Fuel Type Code', 'AER Fuel Type Code', 'Reserved.1',
       'Reserved.2', 'Physical Unit Label', 'Quantity January',
       'Quantity February', 'Quantity March', 'Quantity April', 'Quantity May',
       'Quantity June', 'Quantity July', 'Quantity August',
       'Quantity September', 'Quantity October', 'Quantity November',
       'Quantity December', 'Elec Quantity January', 'Elec Quantity February',
       'Elec Quantity March', 'Elec Quantity April', 'Elec Quantity May',
       'Elec Quantity June', 'Elec Quantity July', 'Elec Quantity August',
       'Elec Quantity September', 'Elec Quantity October',
       'Elec Quantity November', 'Elec Quantity December',
       'MMBtuPer Unit January', 'MMBtuP

In [7]:
#down select columns needed for analysis
eia923g = eia923g_raw[['Plant Id','Combined Heat And Power Plant','Generator Id','Balancing Authority Code','NERC Region', 
        'Census Region','Reported Prime Mover','Net Generation Year To Date','YR']].copy()
eia860 = eia860_raw.copy()
eia860 = eia860[['Plant Code','Generator ID','Technology','Prime Mover','Summer Capacity (MW)','Operating Year','YR']]
eia860 = eia860.rename(columns={'Plant Code':'Plant Id','Generator ID':'Generator Id','Prime Mover':'Reported Prime Mover'})

#create coal worksheet, combine the 860 and 923 data
coal = eia860[eia860['Technology']=='Conventional Steam Coal']
print(coal.shape)
coal_combo = pd.merge(eia923g,coal,on=['Plant Id','Generator Id','Reported Prime Mover','YR'],how='right')
print(coal_combo.shape)
coal_combo.to_csv('outputs/coal.csv',index=False)
print()

#create ngcc worksheet, combine the 860 and 923 data
ngcc = eia860[eia860['Technology']=='Natural Gas Fired Combined Cycle']
print(ngcc.shape)
ngcc_combo = pd.merge(eia923g,ngcc,on=['Plant Id','Generator Id','Reported Prime Mover','YR'],how='right')
print(ngcc_combo.shape)
ngcc_combo.to_csv('outputs/ngcc.csv',index=False)


(6285, 7)
(6285, 12)

(15232, 7)
(15232, 12)


In [8]:
#print(eia923p_raw.columns)
eia923p = eia923p_raw[['Plant Id', 'Reported Prime Mover', 'Reported Fuel Type Code', 'AER Fuel Type Code', 
                       'Elec Fuel Consumption MMBtu', 'Net Generation (Megawatthours)', 'YR',]].copy()
print(eia923p.shape)

#down select to coal steam turbines
coal_p = eia923p[eia923p['Reported Prime Mover']=='ST'].copy()
print(coal_p.shape)
coal_p = coal_p[coal_p['AER Fuel Type Code']=='COL']
print(coal_p.shape)
coal_p = coal_p.drop(columns=['Reported Prime Mover','Reported Fuel Type Code','AER Fuel Type Code'])
coal_p = coal_p.groupby(['Plant Id','YR'],as_index=False).sum()
print(coal_p.shape)
coal_p = coal_p[coal_p['Net Generation (Megawatthours)']>0]
print(coal_p.shape)
print()

#add to the 860 generator data the retired generators to get all units operating throughout the year
eia860_RE = eia860_RE_raw[eia860_RE_raw['Retirement Year']==eia860_RE_raw['YR'].astype(str)].copy()
eia860_op = pd.concat([eia860_raw,eia860_RE])
print(eia860_op.shape)

#down select to steam turbines
coal_g = eia860_op[eia860_op['Prime Mover']=='ST']
print(coal_g.shape)
coal_g = coal_g[['Plant Code','Summer Capacity (MW)','Operating Year','YR']].rename(columns={'Plant Code':'Plant Id'})
print(coal_g.shape)
print()

#aggregate up to plant
coal_g['Operating Year'] = coal_g['Operating Year'].apply(np.int64)
coal_g['Summer Capacity (MW)'] = pd.to_numeric(coal_g['Summer Capacity (MW)'], downcast='float', errors='coerce')
coal_g = coal_g[coal_g['Summer Capacity (MW)']>0]
coal_g['Operating Year'] = coal_g['Operating Year'] * coal_g['Summer Capacity (MW)']
coal_g = coal_g.groupby(['Plant Id','YR'],as_index=False).sum()

#calculate capacity weighted operating year and age of plant
coal_g['Operating Year'] = coal_g['Operating Year'] / coal_g['Summer Capacity (MW)']
coal_g['Operating Year'] = coal_g['Operating Year'].apply(np.int64)
coal_g['Age'] = coal_g['YR'] - coal_g['Operating Year']
#print(coal_g.dtypes)

#add capacity, op year, and age data to 923 dataframe
coal_p = pd.merge(coal_p,coal_g,on=['Plant Id','YR'],how='left')
print(coal_p.shape)
coal_p.to_csv('outputs/heatrate.csv',index=False)


(110252, 7)
(25251, 7)
(5621, 7)
(4223, 4)
(3122, 4)

(177118, 77)
(18696, 77)
(18696, 4)

(3122, 7)
